<a href="https://colab.research.google.com/github/HiroshiATakano/GitHub/blob/master/3%E7%AB%A0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [15]:
import os

path = os.getcwd()

print(path)

try:
    os.chdir(path + "/drive/My Drive/実践データ分析_100本ノック")
except:
    pass

os.getcwd()

/content/drive/My Drive/実践データ分析_100本ノック


'/content/drive/My Drive/実践データ分析_100本ノック'

In [16]:
!pip install japanize-matplotlib

In [17]:
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import japanize_matplotlib

%matplotlib inline

In [89]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactiveity = "all"


In [90]:
!ls ./Data/

10章  7章		     customer_master.csv       transaction_2.csv
1章   8章		     dump_data.csv	       transaction_detail_1.csv
2章   9章		     item_master.csv	       transaction_detail_2.csv
3章   campaign_master.csv    kokyaku_daicho.xlsx       uriage.csv
4章   class_master.csv	     Sample.gsheet	       use_log.csv
5章   customer_join.csv      SpreadsheetSample.gsheet
6章   customer_master_3.csv  transaction_1.csv


In [91]:
import glob

for f in glob.glob('./Data/3章/*.csv'):
  print(os.path.split(f)[1])

customer_master.csv
class_master.csv
campaign_master.csv
use_log.csv


In [92]:
import pathlib

d = {}

p_temp = pathlib.Path('./Data/3章').glob('*.csv')
for p in p_temp:
  f = p.name.split('.')[0] 
  df = pd.read_csv('./Data/3章/' + p.name)
  d[f]= df
  

In [93]:
dk = list(d.keys())
dk[0]

'customer_master'

In [98]:
uselog = d['use_log']
customer = d['customer_master']
class_master = d['class_master']
campaign_master = d['campaign_master']
display(customer.head())
display(customer.isnull().sum())
len(customer)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


customer_id       0
name              0
class             0
gender            0
start_date        0
end_date       2842
campaign_id       0
is_deleted        0
dtype: int64

4192

#ノック２２：顧客データを整形しよう


In [95]:
customer_join = pd.merge(customer, class_master, on="class", how='left')
customer_join = pd.merge(customer_join, campaign_master, on ="campaign_id", how="left")
display(customer_join.head())
len(customer_join)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,ナイト,6000,通常


4192

In [88]:
customer_join.isnull().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

#ノック２３：顧客データの基礎集計をしよう


In [101]:
customer_join.groupby("class_name").count()["customer_id"]

class_name
オールタイム    2045
デイタイム     1019
ナイト       1128
Name: customer_id, dtype: int64

In [102]:
customer_join.groupby("campaign_name").count()["customer_id"]

campaign_name
入会費半額     650
入会費無料     492
通常       3050
Name: customer_id, dtype: int64

In [103]:
customer_join.groupby("gender").count()["customer_id"]

gender
F    1983
M    2209
Name: customer_id, dtype: int64

In [104]:
customer_join.groupby("is_deleted").count()["customer_id"]

is_deleted
0    2842
1    1350
Name: customer_id, dtype: int64

In [107]:
customer_join['start_date'] = pd.to_datetime(customer_join['start_date'])

In [108]:
customer_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4192 entries, 0 to 4191
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   customer_id    4192 non-null   object        
 1   name           4192 non-null   object        
 2   class          4192 non-null   object        
 3   gender         4192 non-null   object        
 4   start_date     4192 non-null   datetime64[ns]
 5   end_date       1350 non-null   object        
 6   campaign_id    4192 non-null   object        
 7   is_deleted     4192 non-null   int64         
 8   class_name     4192 non-null   object        
 9   price          4192 non-null   int64         
 10  campaign_name  4192 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 393.0+ KB


In [112]:
customer_start = customer_join.loc[customer_join["start_date"]>pd.to_datetime("20180401")]
len(customer_start)

1361

In [113]:
customer_start.groupby("is_deleted").count()["customer_id"]

is_deleted
0    716
1    645
Name: customer_id, dtype: int64

In [114]:
customer_start.groupby("campaign_name").count()["customer_id"]

campaign_name
入会費半額    359
入会費無料    317
通常       685
Name: customer_id, dtype: int64

In [124]:
camp = pd.get_dummies(customer_start['campaign_name'])
customer_start_ = pd.concat([customer_start,camp],axis=1)
customer_start_

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,入会費半額,入会費無料,通常
2831,HI195873,XXX,C01,F,2018-04-02,2018-08-31 00:00:00,CA3,1,オールタイム,10500,入会費無料,0,1,0
2832,TS322485,XX,C03,F,2018-04-02,2018-08-31 00:00:00,CA3,1,ナイト,6000,入会費無料,0,1,0
2833,IK552971,XXXXX,C02,F,2018-04-02,2018-08-31 00:00:00,CA3,1,デイタイム,7500,入会費無料,0,1,0
2834,PL000560,XXX,C01,M,2018-04-02,2018-07-31 00:00:00,CA3,1,オールタイム,10500,入会費無料,0,1,0
2835,HD218970,XXXXX,C01,F,2018-04-02,2018-11-30 00:00:00,CA3,1,オールタイム,10500,入会費無料,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,0,0,1
4188,HD246549,XXXXX,C01,F,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,0,0,1
4189,GD037007,XXXXX,C03,M,2019-03-14,NaN,CA1,0,ナイト,6000,通常,0,0,1
4190,OA953150,XXXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,0,0,1


In [125]:
graph_data = customer_start_.set_index('start_date')
graph_data.resample('M').sum()

,is_deleted,price,入会費半額,入会費無料,通常
start_date,,,,,
2018-04-30,132,1377000,0,160,0
2018-05-31,145,1651500,193,0,0
2018-06-30,116,1446000,166,0,0
2018-07-31,81,1140000,0,0,135
2018-08-31,52,877500,0,0,102
2018-09-30,38,762000,0,0,88
2018-10-31,22,531000,0,0,66
2018-11-30,17,556500,0,0,65
2018-12-31,28,1339500,0,157,0
